# Version 3

- Now has a switch for plotting all the images, or not.

- Does not read in the master files if they already exist. 

- The histograms are now better and have 50 bins spread over a more useful range. 

- Only divides by 16 for the ASI1600MM Camera. 

- Better file handling, although it does mean the files have to be in the right place. 

- 2//12/2023 PXS

# Version 4

- Now actually has the divide by 16 for only the 1600MM camera as V3 didn't have it for some odd reason

- Now also update scale to the new binned data, so at least for the binned image the scale etc. are correct for that image.

# Version 5

- Asume a certain file structure. Get the colours and then run through all the images to make reduced images. Will cut out much faffing about. TO DO

# Version 6

- Seciton at end to make a saturation mask. DONE

# Version 7

- A bust did not use

# Version 8

- Save redcued images to a subfolder and the saturated images also to a subfolder. DONE
- Write file path into the header of the raw images. DONE

Here I'll make my fist attempt to open and look at my own data. using the tutorial from [here](https://learn.astropy.org/tutorials/FITS-images.html)

In [1]:
import numpy as np

# Set up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from astropy.io import fits
from astropy.utils.data import download_file

import os
import glob
import time
from matplotlib.colors import LogNorm
from IPython import display
from IPython.display import clear_output
import random

from astropy.nddata import block_reduce

from My_Functions_v3 import create_or_empty_folder
from My_Functions_v3 import update_fits_header

plt.rcParams.update({'font.size':15})

In [2]:
plotting = True
plotting = False

Next want to grab all the file names in a sub-folder. I'm using glob to do it as it saves me typing in the file names by hand.

In [3]:
file_object = "HeartNebula"
file_night  = "2023_11_30_apo"

In [3]:
file_object  = "tyc_3023-1974-1"

file_night  = "2023_04_02"
#file_night  = "2023_04_03"

In [121]:
file_object = "RZ_Cep"
file_night  = "2022_12_07"
file_night  = "2023_01_23"
file_night  = "2023_02_23"
file_night  = "2023_11_10"
file_night  = "2024_01_07"
file_night  = "2024_01_18"

In [249]:
file_object = "RZ_Cas"
file_night  = "2023_03_02"
file_night  = "2023_04_06"
file_night  = "2023_10_14"
file_night  = "2023_11_30"

In [250]:
binned = True #2x2
Single = True  #instead of doubles

In [251]:
colour = "Luminance"
colour = "Red"
colour = "Green"
colour = "Blue"
#colour = "OIII"
#colour = "Ha"
#colour = "SII"

In [252]:
filepath     = "../../BPO_Data/"+file_object+"/"+file_night+"/Light/"+colour+"/"
flatfilepath = "../../BPO_Data/"+file_object+"/"+file_night+"/Flat/"+colour+"/master_flat.fits"
barkfilepath = "../../BPO_Data/"+file_object+"/"+file_night+"/Dark/master_dark_per_sec.fits"
biasfilepath = "../../BPO_Data/"+file_object+"/"+file_night+"/Bias/master_bias.fits"

main_star = file_object

In [253]:
image_list = []

for file in glob.glob(filepath+"*.fits"): #The 'D' stops it picking up past master_frames
        image_list.append(file)
    
print("\n".join(image_list))


../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_001.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_002.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_003.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_004.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_005.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_006.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_007.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_008.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_009.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_010.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_011.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_012.fits
../../BPO_Data/R

In [254]:
masterbias_image = fits.getdata(biasfilepath)

In [255]:
if plotting == True:
    plt.figure(figsize=(15,6))

    N_std = 1.5

    print('Min:', np.min(masterbias_image) )
    print('Max:', np.max(masterbias_image)  )    
    print('Mean:', np.mean(masterbias_image)  )
    print('Median', np.median(masterbias_image)  )
    print('Stdev:', np.std(masterbias_image)  )

    Vmin = np.median(masterbias_image) - N_std*np.std(masterbias_image)
    Vmax = np.median(masterbias_image) + N_std*np.std(masterbias_image)

    plt.subplot(1,2,1)
    plt.imshow(masterbias_image, cmap='gray', 
                   vmin=Vmin , vmax=Vmax) 
    plt.title("Master Bias")
    plt.colorbar()


    plt.subplot(1,2,2)
    plt.hist(masterbias_image.flatten(),bins=50)
    plt.yscale('log')

    plt.show()

In [256]:
masterdark_image = fits.getdata(barkfilepath)

In [257]:
if plotting == True:
    plt.figure(figsize=(15,6))

    N_std = 1.5

    print('Min:', np.min(masterdark_image) )
    print('Max:', np.max(masterdark_image)  )    
    print('Mean:', np.mean(masterdark_image)  )
    print('Median', np.median(masterdark_image)  )
    print('Stdev:', np.std(masterdark_image)  )

    Vmin = np.median(masterdark_image) - N_std*np.std(masterdark_image)
    Vmax = np.median(masterdark_image) + N_std*np.std(masterdark_image)

    plt.subplot(1,2,1)
    plt.imshow(masterdark_image, cmap='gray', 
                   vmin= Vmin, vmax=Vmax) 
    plt.title("Master Dark")
    plt.colorbar()


    plt.subplot(1,2,2)
    plt.hist(masterdark_image.flatten(),bins=50, range=(Vmin,Vmax))
    plt.yscale('log')

    plt.show()

In [258]:
master_flat_image = fits.getdata(flatfilepath)

In [259]:
if plotting == True:
    plt.figure(figsize=(15,6))

    N_std = 3.5

    print('Min:', np.min(master_flat_image) )
    print('Max:', np.max(master_flat_image)  )    
    print('Mean:', np.mean(master_flat_image)  )
    print('Median', np.median(master_flat_image)  )
    print('Stdev:', np.std(master_flat_image)  )

    Vmin = np.median(master_flat_image) - N_std*np.std(master_flat_image)
    Vmax = Vmin + 2*N_std*np.std(master_flat_image)
    plt.subplot(1,2,1)
    plt.imshow(master_flat_image, cmap='gray', 
                   vmin=Vmin, vmax=Vmax) 
    plt.title("Master Flat")
    plt.colorbar()


    plt.subplot(1,2,2)
    plt.hist(master_flat_image.flatten(),bins=50,range=(Vmin,Vmax))
    plt.yscale('log')

    plt.show()

See if a subfolder exists, and if it does empty it!

In [260]:
create_or_empty_folder(filepath+"reduced_images/")

Folder '../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_images/' has been created.


In [261]:
i=1
for file in image_list:
    clear_output(wait=True)
    print(str(i) + "/" + str(len(image_list)))
    print(file)
    #clear_output(wait=True)
    #plt.figure(figsize=(15,6))
    
    hdul = fits.open(file)
    
    reduced_image = []
    raw_image = []
    raw_image = hdul[0].data 
    
    
    image_hdr = []
    image_hdr = hdul[0].header
    
    # remove factor of 16 only for our 12 bit camera
    if image_hdr['INSTRUME'] == 'ZWO CCD ASI1600MM Pro':
        raw_image = raw_image / 16
        
    exposure = image_hdr["EXPTIME "]
    #print(exposure)
    
    reduced_image = (raw_image - masterdark_image*exposure - masterbias_image) / master_flat_image
    # get to flux.
    reduced_image = reduced_image / exposure
    if Single == 1:
        reduced_image = reduced_image.astype('float32')
    
    if binned == 1:
        reduced_image = block_reduce(reduced_image, block_size=(2, 2), func=np.sum)
        
    file_name = os.path.basename(file)
    new_file_name = filepath + "reduced_images/" + file_name.replace('.fits', '_reduced.fits')

    hdul[0] = fits.PrimaryHDU(reduced_image,image_hdr)
    hdul[0].header['REDTIME'] = 1
    if binned == 1:
        hdul[0].header['XBINNING'] = 2
        hdul[0].header['YBINNING'] = 2
        hdul[0].header['SCALE'] = 2 *  hdul[0].header['SCALE']

    hdul.writeto(new_file_name, overwrite=True)
    
    # Store the file name in the orignal fits file
    update_fits_header(file, 'RedImage', new_file_name, comment="See Data_Reduction_v8t")
    
    
    i=i+1
    time.sleep(0.1) 
    
    #time.sleep(1) #pause in secs. Make bigger if you need.    

50/50
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_050.fits


Now we'll make another file that has an image of if the pixels were saturated or not.

In [262]:
reduced_image_list = []

for file in image_list: 
    hdr_temp = fits.getheader(file)
    reduced_image_list.append(hdr_temp["REDIMAGE"])


print("\n".join(reduced_image_list))

../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_images/RZ_Cas_Light_Blue_0.600_secs_001_reduced.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_images/RZ_Cas_Light_Blue_0.600_secs_002_reduced.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_images/RZ_Cas_Light_Blue_0.600_secs_003_reduced.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_images/RZ_Cas_Light_Blue_0.600_secs_004_reduced.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_images/RZ_Cas_Light_Blue_0.600_secs_005_reduced.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_images/RZ_Cas_Light_Blue_0.600_secs_006_reduced.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_images/RZ_Cas_Light_Blue_0.600_secs_007_reduced.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_images/RZ_Cas_Light_Blue_0.600_secs_008_reduced.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_images/RZ_Cas_Light_Blue_0.600_secs_009_reduced.fits
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/reduced_ima

In [263]:
create_or_empty_folder(filepath+"saturated_images/")

Folder '../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue/saturated_images/' has been created.


In [264]:
i=0
for file in image_list:
    
    clear_output(wait=True)
    print(str(i+1) + "/" + str(len(image_list)))
    print(file)
    
    # Get the reduced image header
    hdul = fits.open(reduced_image_list[i])
    image_hdr = hdul[0].header
    
    if image_hdr["XBINNING"] == 2:
        binned = 1
    
    raw_image = fits.getdata(file, 0)
    
    
    #### Saturated?
    threshold = 0.9 * 2**16
    above_threshold = raw_image < threshold


    
    if binned == 1:
        above_threshold = block_reduce(above_threshold, block_size=(2, 2), func=np.prod)
  

    hdul[0] = fits.PrimaryHDU(above_threshold,image_hdr)
    
    # Save the modified FITS file
    
    file_name = os.path.basename(file)
    new_file_name = filepath + "saturated_images/" + file_name.replace('.fits', '_saturated.fits')
        
    hdul.writeto(new_file_name, overwrite=True)
    hdul.close()
    
    # Store the file name in the orignal fits file
    update_fits_header(file, 'SatImage', new_file_name, comment="See Data_Reduction_v8t")
    
    i=i+1
    
    
    time.sleep(0.1) 
    
    #time.sleep(1) #pause in secs. Make bigger if you need.    

50/50
../../BPO_Data/RZ_Cas/2023_11_30/Light/Blue\RZ_Cas_Light_Blue_0.600_secs_050.fits
